Import Libraries

In [1]:
import os
import msoffcrypto
import pandas as pd
import numpy as np
from datetime import datetime as dt
from matplotlib import pyplot as plt
from scipy.interpolate import CubicSpline
from sklearn.linear_model import LinearRegression
import pmdarima as pm
from pmdarima import model_selection
import math
import pytz
import datetime

In [2]:
xls = pd.ExcelFile("RawData13.xlsx")
Factor_data = pd.read_excel(xls,'Factor')
Gold_data = pd.read_excel(xls,'GoldBar_Sale')

In [3]:
factor_col = Factor_data.columns[1:]

In [4]:
factor_col

Index(['(Inflation)', 'PTT_แก๊สโซฮอล์_95', 'PTT_แก๊สโซฮอล์_91', 'PTT_ดีเซล',
       'อัตราแลกเปลี่ยน_USD_THB', 'BOT_Policy_rate_(อัตราดอกเบี้ย นโยบาย)',
       'SCB_ออมทรัพย์(อัตราดอกเบี้ย)', 'SCB_ประจำ_3_เดือน(อัตราดอกเบี้ย)',
       'SCB_ประจำ_6_เดือน(อัตราดอกเบี้ย)', 'SCB_ประจำ_12_เดือน(อัตราดอกเบี้ย)',
       'XAU_USD'],
      dtype='object')

In [5]:
for month in range(1,9):
    Factor_data[[i+f'_LAG_{month}' for i in factor_col]] = Factor_data[factor_col].shift(month)

In [6]:
Factor_data.columns

Index(['Report Date', '(Inflation)', 'PTT_แก๊สโซฮอล์_95', 'PTT_แก๊สโซฮอล์_91',
       'PTT_ดีเซล', 'อัตราแลกเปลี่ยน_USD_THB',
       'BOT_Policy_rate_(อัตราดอกเบี้ย นโยบาย)',
       'SCB_ออมทรัพย์(อัตราดอกเบี้ย)', 'SCB_ประจำ_3_เดือน(อัตราดอกเบี้ย)',
       'SCB_ประจำ_6_เดือน(อัตราดอกเบี้ย)', 'SCB_ประจำ_12_เดือน(อัตราดอกเบี้ย)',
       'XAU_USD', '(Inflation)_LAG_1', 'PTT_แก๊สโซฮอล์_95_LAG_1',
       'PTT_แก๊สโซฮอล์_91_LAG_1', 'PTT_ดีเซล_LAG_1',
       'อัตราแลกเปลี่ยน_USD_THB_LAG_1',
       'BOT_Policy_rate_(อัตราดอกเบี้ย นโยบาย)_LAG_1',
       'SCB_ออมทรัพย์(อัตราดอกเบี้ย)_LAG_1',
       'SCB_ประจำ_3_เดือน(อัตราดอกเบี้ย)_LAG_1',
       'SCB_ประจำ_6_เดือน(อัตราดอกเบี้ย)_LAG_1',
       'SCB_ประจำ_12_เดือน(อัตราดอกเบี้ย)_LAG_1', 'XAU_USD_LAG_1',
       '(Inflation)_LAG_2', 'PTT_แก๊สโซฮอล์_95_LAG_2',
       'PTT_แก๊สโซฮอล์_91_LAG_2', 'PTT_ดีเซล_LAG_2',
       'อัตราแลกเปลี่ยน_USD_THB_LAG_2',
       'BOT_Policy_rate_(อัตราดอกเบี้ย นโยบาย)_LAG_2',
       'SCB_ออมทรัพย์(อัตราดอกเบี้ย)_LAG_2'

In [7]:
# Factor_data[['LOG_'+i for i in factor_col]] = \
#     Factor_data[[i for i in factor_col]].apply(np.log)
for month in range(0,2):
    Factor_data[['LOG_'+i+f'_LAG_{month}' if month!=0 else 'LOG_'+i for i in factor_col]] = \
    Factor_data[[i+f'_LAG_{month}' if month!=0 else i for i in factor_col]].apply(np.log)

C:\Users\temp5\AppData\Local\Temp\ipykernel_23248\563302744.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Factor_data[['LOG_'+i+f'_LAG_{month}' if month!=0 else 'LOG_'+i for i in factor_col]] = \
C:\Users\temp5\AppData\Local\Temp\ipykernel_23248\563302744.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Factor_data[['LOG_'+i+f'_LAG_{month}' if month!=0 else 'LOG_'+i for i in factor_col]] = \
C:\Users\temp5\AppData\Local\Temp\ipykernel_23248\563302744.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usu

In [8]:
Factor_data.columns

Index(['Report Date', '(Inflation)', 'PTT_แก๊สโซฮอล์_95', 'PTT_แก๊สโซฮอล์_91',
       'PTT_ดีเซล', 'อัตราแลกเปลี่ยน_USD_THB',
       'BOT_Policy_rate_(อัตราดอกเบี้ย นโยบาย)',
       'SCB_ออมทรัพย์(อัตราดอกเบี้ย)', 'SCB_ประจำ_3_เดือน(อัตราดอกเบี้ย)',
       'SCB_ประจำ_6_เดือน(อัตราดอกเบี้ย)',
       ...
       'LOG_PTT_แก๊สโซฮอล์_95_LAG_1', 'LOG_PTT_แก๊สโซฮอล์_91_LAG_1',
       'LOG_PTT_ดีเซล_LAG_1', 'LOG_อัตราแลกเปลี่ยน_USD_THB_LAG_1',
       'LOG_BOT_Policy_rate_(อัตราดอกเบี้ย นโยบาย)_LAG_1',
       'LOG_SCB_ออมทรัพย์(อัตราดอกเบี้ย)_LAG_1',
       'LOG_SCB_ประจำ_3_เดือน(อัตราดอกเบี้ย)_LAG_1',
       'LOG_SCB_ประจำ_6_เดือน(อัตราดอกเบี้ย)_LAG_1',
       'LOG_SCB_ประจำ_12_เดือน(อัตราดอกเบี้ย)_LAG_1', 'LOG_XAU_USD_LAG_1'],
      dtype='object', length=122)

In [9]:
# ln(self/lag) -> lag 12 months
# for month in range(1,13):
#     Factor_data[['LOG_('+i+f'/{i}_LAG_{month})' for i in factor_col]] = \
#     np.log(Factor_data[[i for i in factor_col]].values/Factor_data[[f'{i}_LAG_{month}' for i in factor_col]].values)

In [10]:
# Factor_data.columns

In [11]:
# ln(self/lag) MoM lag 1-12 months
# for month in range(1,13):
#     Factor_data[['LOG_MoM_('+i+f'/{i}_LAG_1)_LAG_{month}' for i in factor_col]] = Factor_data[['LOG_('+i+f'/{i}_LAG_1)' for i in factor_col]].shift(month)

In [12]:
# Factor_data.columns

In [13]:
Gold_Factor_data = \
                pd.merge(
                    Gold_data,
                    Factor_data,
                    left_on = 'Report Date',
                    right_on = 'Report Date',
                    how = 'left'
                )

In [22]:
Gold_Factor_data.replace([np.inf, -np.inf, np.nan],0,inplace=True)
# Gold_Factor_data.replace([np.inf, -np.inf],np.nan,inplace=True)
Gold_Factor_data = Gold_Factor_data.dropna(axis='columns')

In [23]:
Gold_Factor_data.columns

Index(['Report Date', 'GoldBar_Sale', '(Inflation)', 'PTT_แก๊สโซฮอล์_95',
       'PTT_แก๊สโซฮอล์_91', 'PTT_ดีเซล', 'อัตราแลกเปลี่ยน_USD_THB',
       'BOT_Policy_rate_(อัตราดอกเบี้ย นโยบาย)',
       'SCB_ออมทรัพย์(อัตราดอกเบี้ย)', 'SCB_ประจำ_3_เดือน(อัตราดอกเบี้ย)',
       ...
       'LOG_PTT_แก๊สโซฮอล์_91_LAG_1', 'LOG_PTT_ดีเซล_LAG_1',
       'LOG_อัตราแลกเปลี่ยน_USD_THB_LAG_1',
       'LOG_BOT_Policy_rate_(อัตราดอกเบี้ย นโยบาย)_LAG_1',
       'LOG_SCB_ออมทรัพย์(อัตราดอกเบี้ย)_LAG_1',
       'LOG_SCB_ประจำ_3_เดือน(อัตราดอกเบี้ย)_LAG_1',
       'LOG_SCB_ประจำ_6_เดือน(อัตราดอกเบี้ย)_LAG_1',
       'LOG_SCB_ประจำ_12_เดือน(อัตราดอกเบี้ย)_LAG_1', 'LOG_XAU_USD_LAG_1',
       'LOGIT'],
      dtype='object', length=124)

In [38]:
Gold_Factor_data = Gold_Factor_data[Gold_Factor_data['Report Date'] >= '2021-01-31']
Gold_Factor_data = Gold_Factor_data[Gold_Factor_data['Report Date'] <= '2023-12-31']

In [39]:
Gold_Factor_data

,Report Date,GoldBar_Sale,(Inflation),PTT_แก๊สโซฮอล์_95,PTT_แก๊สโซฮอล์_91,PTT_ดีเซล,อัตราแลกเปลี่ยน_USD_THB,BOT_Policy_rate_(อัตราดอกเบี้ย นโยบาย),SCB_ออมทรัพย์(อัตราดอกเบี้ย),SCB_ประจำ_3_เดือน(อัตราดอกเบี้ย),...,LOG_PTT_แก๊สโซฮอล์_91_LAG_1,LOG_PTT_ดีเซล_LAG_1,LOG_อัตราแลกเปลี่ยน_USD_THB_LAG_1,LOG_BOT_Policy_rate_(อัตราดอกเบี้ย นโยบาย)_LAG_1,LOG_SCB_ออมทรัพย์(อัตราดอกเบี้ย)_LAG_1,LOG_SCB_ประจำ_3_เดือน(อัตราดอกเบี้ย)_LAG_1,LOG_SCB_ประจำ_6_เดือน(อัตราดอกเบี้ย)_LAG_1,LOG_SCB_ประจำ_12_เดือน(อัตราดอกเบี้ย)_LAG_1,LOG_XAU_USD_LAG_1,LOGIT
120,2021-01-31,26250,0.21,24.25,23.98,25.09,29.86,0.50,0.25,0.375,...,3.112626,3.185939,3.402530,-0.693147,-1.386294,-0.980829,-0.693147,-0.693147,7.160124,10.175421
121,2021-02-28,25350,0.04,26.05,25.78,26.99,30.47,0.50,0.25,0.370,...,3.177220,3.222469,3.396520,-0.693147,-1.386294,-0.980829,-0.693147,-0.693147,7.156247,10.140534
122,2021-03-31,25500,0.09,26.15,25.88,23.19,31.24,0.50,0.25,0.320,...,3.249599,3.295466,3.416743,-0.693147,-1.386294,-0.994252,-0.798508,-0.798508,7.190819,10.146434
123,2021-04-30,26250,0.30,29.05,28.78,25.99,31.14,0.50,0.25,0.320,...,3.253470,3.143721,3.441699,-0.693147,-1.386294,-1.139434,-0.916291,-0.916291,7.131451,10.175421
124,2021-05-31,28100,0.49,27.95,27.68,25.09,31.19,0.50,0.25,0.320,...,3.359681,3.257712,3.438493,-0.693147,-1.386294,-1.139434,-0.916291,-0.916291,7.163397,10.243525
125,2021-06-30,26900,0.52,28.75,28.48,25.79,32.02,0.50,0.25,0.320,...,3.320710,3.222469,3.440098,-0.693147,-1.386294,-1.139434,-0.916291,-0.916291,7.157455,10.199882
126,2021-07-31,28300,0.14,29.55,29.28,26.09,32.89,0.50,0.25,0.320,...,3.349202,3.249987,3.466361,-0.693147,-1.386294,-1.139434,-0.916291,-0.916291,7.189764,10.250617
127,2021-08-30,27950,0.07,29.25,28.98,25.79,32.22,0.50,0.25,0.320,...,3.376905,3.261552,3.493169,-0.693147,-1.386294,-1.139434,-0.916291,-0.916291,7.125436,10.238172
128,2021-09-30,27850,0.19,30.75,30.48,27.69,33.67,0.50,0.25,0.320,...,3.366606,3.249987,3.472587,-0.693147,-1.386294,-1.139434,-0.916291,-0.916291,7.094185,10.234588
129,2021-10-31,28200,0.21,32.55,32.28,29.69,33.29,0.50,0.25,0.320,...,3.417071,3.321071,3.516607,-0.693147,-1.386294,-1.139434,-0.916291,-0.916291,7.132490,10.247077


In [40]:
Gold_Factor_data['LOGIT'] = np.log(Gold_Factor_data['GoldBar_Sale'].values)
Gold_Factor_data['LOGIT'].replace([np.inf, -np.inf, np.nan],0,inplace=True)

C:\Users\temp5\AppData\Local\Temp\ipykernel_23248\3363949668.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Gold_Factor_data['LOGIT'].replace([np.inf, -np.inf, np.nan],0,inplace=True)


In [41]:
Factor_data.columns

Index(['Report Date', '(Inflation)', 'PTT_แก๊สโซฮอล์_95', 'PTT_แก๊สโซฮอล์_91',
       'PTT_ดีเซล', 'อัตราแลกเปลี่ยน_USD_THB',
       'BOT_Policy_rate_(อัตราดอกเบี้ย นโยบาย)',
       'SCB_ออมทรัพย์(อัตราดอกเบี้ย)', 'SCB_ประจำ_3_เดือน(อัตราดอกเบี้ย)',
       'SCB_ประจำ_6_เดือน(อัตราดอกเบี้ย)',
       ...
       'LOG_PTT_แก๊สโซฮอล์_95_LAG_1', 'LOG_PTT_แก๊สโซฮอล์_91_LAG_1',
       'LOG_PTT_ดีเซล_LAG_1', 'LOG_อัตราแลกเปลี่ยน_USD_THB_LAG_1',
       'LOG_BOT_Policy_rate_(อัตราดอกเบี้ย นโยบาย)_LAG_1',
       'LOG_SCB_ออมทรัพย์(อัตราดอกเบี้ย)_LAG_1',
       'LOG_SCB_ประจำ_3_เดือน(อัตราดอกเบี้ย)_LAG_1',
       'LOG_SCB_ประจำ_6_เดือน(อัตราดอกเบี้ย)_LAG_1',
       'LOG_SCB_ประจำ_12_เดือน(อัตราดอกเบี้ย)_LAG_1', 'LOG_XAU_USD_LAG_1'],
      dtype='object', length=122)

In [42]:
Gold_Factor_data['LOGIT']

120    10.175421
121    10.140534
122    10.146434
123    10.175421
124    10.243525
125    10.199882
126    10.250617
127    10.238172
128    10.234588
129    10.247077
130    10.262909
131    10.262909
132    10.255903
133    10.280210
134    10.327120
135    10.335270
136    10.308953
137    10.322198
138    10.315597
139    10.300584
140    10.288750
141    10.297217
142    10.298902
143    10.297217
144    10.307285
145    10.308953
146    10.367222
147    10.375052
148    10.373491
149    10.378168
150    10.367222
151    10.373491
152    10.393600
153    10.439981
154    10.422281
155    10.423768
Name: LOGIT, dtype: float64

In [43]:
Gold_Factor_data.to_excel("LAG_RESULT.xlsx")

เลือกตัวแปรที่สนใจ และเกี่ยวข้องกับทองคำ

In [44]:
import statsmodels.api as sm
from statsmodels.compat import lzip
from statsmodels.tsa.stattools import adfuller
main_Factor_list = factor_col
# Factor_list = Gold_Factor_data.drop(columns = ['GoldBar_Sale'] ).columns
# Factor_list = Gold_Factor_data.drop(columns = ['Report Date','GoldBar_Sale','LOGIT']).columns
Factor_list = Gold_Factor_data.drop(columns = ['Report Date','GoldBar_Sale','LOGIT']).columns
All_result_table = pd.DataFrame()
All_result_table_best = pd.DataFrame()

for ix , temp_factor_group in enumerate(main_Factor_list,1):
    Factor_group = [i for i in Factor_list if temp_factor_group in i]
    r2_list = []
    # print(ix , " " , temp_factor_group , " " , Factor_group)
    # print(Factor_group)
    temp_result_table = pd.DataFrame()
    for temp_factor in Factor_group:
        X_train = np.array(Gold_Factor_data[temp_factor].values , dtype=float)
        y_train = np.array(Gold_Factor_data['LOGIT'].values , dtype=float)
        # y_train = np.array(Gold_Factor_data['GoldBar_Sale'].values , dtype=float)
        
        X_train_1 = X_train.reshape(-1,1)
        y_train_1 = y_train.reshape(-1,1)

        # test stationary test 10%
        adf = adfuller(X_train)[1]
        adf_result = 'PASS' if adfuller(X_train)[1] <= 0.4 else 'FAIL'

        #Regression
        linear_model = LinearRegression()
        linear_model.fit(X_train_1 , y_train_1)

        X_train = sm.add_constant(X_train)
        reg = sm.OLS(y_train , X_train).fit()
        sign = '+' if linear_model.coef_ > 0 else '-'

        r2_list.append((temp_factor,reg.rsquared,sign,adf,adf_result))
        # print(r2_list)

    temp_result_table['Factor'] = np.array(r2_list)[:,0]
    temp_result_table['r2'] = np.array(r2_list)[:,1]
    temp_result_table['sign'] = np.array(r2_list)[:,2]
    temp_result_table['adf'] = np.array(r2_list)[:,3]
    temp_result_table['adf_result'] = np.array(r2_list)[:,4]

    temp_result_table_sort = temp_result_table.sort_values(['adf_result','r2'], ascending = [False,False]).reset_index(drop=True)

    if ix == 1:
        All_result_table = temp_result_table
        All_result_table_best = temp_result_table_sort.head(1)
    else:
        All_result_table = pd.concat([All_result_table,temp_result_table], ignore_index = True)
        All_result_table_best = pd.concat([All_result_table,temp_result_table_sort.head(1)], ignore_index = True)
All_result_table

ValueError: Invalid input, x is constant

In [ ]:
Factor_list

In [ ]:
All_result_table[All_result_table['adf_result'] == 'PASS']

In [ ]:
Pass_Factor_list = All_result_table[All_result_table['adf_result'] == 'PASS']
Pass_Factor_list.to_excel("ADF_RESULT_PASS.xlsx")

In [ ]:
Pass_Factor_list = Pass_Factor_list['Factor']

Overall Model Performance

In [ ]:
from itertools import combinations
# comb_list = [i for i in combinations(Pass_Factor_list , 3)] + [i for i in combinations(Pass_Factor_list, 2)]
comb_list = [i for i in combinations(Pass_Factor_list , 4)]
# comb_list = [i for i in combinations(Pass_Stat_list['FACTOR_1'], 2)]
len(comb_list)
# 2 fac
#5000 = 2 min
#2346 = 50 sec
#30856 = 14 min
#4186 = 1.34 min/sec
#9453 = 3.32 min/sec
#17020 = 6.52 min/sec # 7.45.39 min/sec/msec
# 1 fac
#185 = 12ms
# new 3 fac
# 11480 = 5 min
# 2925 =1 min 10s
# new 4 fac
# 17550 = 9 mins 
# new 3 fac
# 2925 = 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.stats import anderson as AD
from statsmodels.stats.outliers_influence import variance_inflation_factor

all_stats_results = pd.DataFrame()

# y_train = np.array(Gold_Factor_data['GoldBar_Sale'].values , dtype=float)
y_train = np.array(Gold_Factor_data['LOGIT'].values , dtype=float)

for out_index , comb in enumerate(comb_list,1):
    selected_Factor = list(comb)
    X_train = np.array(Gold_Factor_data[selected_Factor].values , dtype = float)

    linear_model = LinearRegression()
    linear_model.fit(X_train , y_train)

    X_train = sm.add_constant(X_train)
    reg = sm.OLS(y_train , X_train).fit()

    X_train = X_train[:,1:]
    y_pred = linear_model.predict(X_train)

    stats_results = pd.DataFrame()
    stats_results[[f'FACTOR_{i+1}' for i in range(len(selected_Factor))]] = np.array([selected_Factor])

    coeff_result = reg.summary2().tables[1].iloc[:,:1].transpose().reset_index(drop=True)
    # for i in ['const','x1']:
    for i in ['const','x1','x2','x3']:
        if i in coeff_result.columns:
            stats_results[i] = coeff_result[i]
        else:
            stats_results[i] = None
    # P-Value
    stats_results[['intercept_p_value'] + [f'FACTOR_{i+1}_p_value' for i in range(len(selected_Factor))]] = reg.pvalues
    # ADJ Rsq
    r2 = r2_score(y_train,y_pred)
    n , p = X_train.shape
    adj_r2 = 1 - (( 1 - r2 ) * (n - 1) / (n - p - 1))
    stats_results['r2'] = r2
    stats_results['adj_r2'] = adj_r2
    # F stats
    stats_results['Probability of F-Stats'] = reg.summary2().tables[0][3].astype(float)[5]
    # Anderson Darling
    stats_results['AD'] = AD(reg.resid , dist='norm')[0]
    stats_results['AD_criteria'] = [{i:j for i , j in zip(AD( reg.resid , dist='norm')[2],
                                                         AD( reg.resid , dist='norm')[1]
                                                         )}]
    stats_results['AD_criteria_0.05'] = AD( reg.resid , dist='norm')[1][2:3]
    # Breusch-Pagan
    name_stats = ['Lagrange multiplier statistic','p-value','f-value','f p-value']
    test = sm.stats.het_breuschpagan(reg.resid , reg.model.exog )
    stats_results['BP_pvalues'] = lzip(name_stats , test )[1][1]
    # Durbin Watson
    stats_results['Durbin_Watson'] = sm.stats.durbin_watson(reg.resid)
    # VIF 
    # VIF_values = [variance_inflation_factor(Gold_Factor_data[selected_Factor].values , i) for i in range(len(selected_Factor))]

    # for in_index , i in enumerate(selected_Factor , 0 ):
    #     stats_results[f'VIF_Factor_{in_index+1}'] = VIF_values[in_index]
    if out_index == 1:
        all_stats_results = stats_results
    else:
        all_stats_results = pd.concat([all_stats_results, stats_results], axis = 0).reset_index(drop=True)
all_stats_results = all_stats_results.sort_values('adj_r2', ascending = False)

In [ ]:
Pass_Stat_list = all_stats_results

In [ ]:
#Adj Stats in factor to interesting
Pass_Stat_list = all_stats_results[all_stats_results['r2'] >= 0.5]
Pass_Stat_list = all_stats_results[all_stats_results['adj_r2'] >= 0.5]
Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['Probability of F-Stats'] <= 0.5]
Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['AD'] < Pass_Stat_list['AD_criteria_0.05']]
# Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['BP_pvalues'] > 0.05]
# Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['Durbin_Watson'] >= 1] # ใช้ไมได้
# Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['VIF_Factor_1'] <= 5]
# Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['VIF_Factor_2'] <= 5]
# Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['VIF_Factor_3'] <= 5]
# Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['VIF_Factor_4'] <= 5]
# Pass_Stat_list = Pass_Stat_list[(Pass_Stat_list['VIF_Factor_3'] <= 5 | (Pass_Stat_list['VIF_Factor_3'].isna()))]

In [ ]:
# Pass_Stat_list['adj_r2'].max()
# Pass_Stat_list[Pass_Stat_list['adj_r2'].values == Pass_Stat_list['adj_r2'].max()]
# Pass_Stat_list.describe()
# Pass_Stat_list.info()
# Pass_Stat_list[['FACTOR_1','FACTOR_2','const','x1','x2','r2','adj_r2','AD','AD_criteria_0.05','BP_pvalues','Durbin_Watson','Probability of F-Stats','VIF_Factor_1','VIF_Factor_2']]
Pass_Stat_list[['FACTOR_1','FACTOR_2','FACTOR_3','const','x1','x2','x3','r2','adj_r2','AD','AD_criteria_0.05','BP_pvalues','Durbin_Watson','Probability of F-Stats','VIF_Factor_1','VIF_Factor_2','VIF_Factor_3']]
Pass_Stat_list[['FACTOR_1','FACTOR_2','FACTOR_3','const','x1','x2','x3','r2','adj_r2','Durbin_Watson','Probability of F-Stats','VIF_Factor_1','VIF_Factor_2','VIF_Factor_3']]
Pass_Stat_list[['FACTOR_1','FACTOR_2','FACTOR_3','const','x1','x2','x3','x4','r2','adj_r2','Durbin_Watson','Probability of F-Stats','VIF_Factor_1','VIF_Factor_2','VIF_Factor_3','VIF_Factor_3']]
# Pass_Stat_list[['FACTOR_1','intercept_p_value','const','x1','r2','adj_r2','AD','AD_criteria_0.05','BP_pvalues','Durbin_Watson','Probability of F-Stats']]


In [ ]:
Pass_Stat_list

In [ ]:
Pass_Stat_list.to_excel('Pass_Stat_list 1 factor.xlsx')

In [ ]:
Pass_Stat_list.to_excel("Pass_Stat_list.xlsx")

In [ ]:
# ตัวแปรที่สนใจ 
Factor_interested = ['(Inflation)_LAG_6','LOG_SCB_ประจำ_6_เดือน(อัตราดอกเบี้ย)_LAG_1']
Factor_value = [9.925955,0.117846,-0.259044]

In [ ]:
Test_Stat =Pass_Stat_list[(Pass_Stat_list['FACTOR_1'] == '(Inflation)_LAG_6') & (Pass_Stat_list['FACTOR_2'] == 'LOG_SCB_ประจำ_6_เดือน(อัตราดอกเบี้ย)_LAG_1')]

In [ ]:
New_Predict = (Test_Stat['const'].values + (Test_Stat['x1'].values * Gold_Factor_data[Test_Stat['FACTOR_1']].values) +\
                     (Test_Stat['x2'].values * (Gold_Factor_data[Test_Stat['FACTOR_2']].values)))
# New_Predict.to_excel('Test.xlsx')
np.exp(New_Predict)

In [ ]:
Prediction_df = pd.DataFrame()
Prediction_df['Report Date'] =Gold_Factor_data['Report Date']
Prediction_df['Predict GoldBar_Sale price (Forward looking)'] = np.exp(New_Predict)
Prediction_df['LOG'] = New_Predict
Prediction_df.to_excel('Prediction.xlsx')

Actual data vs Predict data (GoldBar_Sale price in Thailnad) 2 Factor

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(Gold_Factor_data['Report Date'],np.log(Gold_Factor_data['GoldBar_Sale']),label='Actual Values', color='blue',marker='o')
plt.plot(Prediction_df['Report Date'],np.log(Prediction_df['Predict GoldBar_Sale price (Forward looking)']),label='Predicted Values',
         color='red',linestyle='--',marker='x')
plt.xlabel('Report Date')
plt.ylabel('Value')
plt.title('Actual vs Predict values')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error , mean_squared_error
# Mean Absolute Error (MAE)
MAE = mean_absolute_error(np.log(Gold_Factor_data['GoldBar_Sale']),np.log(Prediction_df['Predict GoldBar_Sale price (Forward looking)']))
# Mean Squared Error (MSE)
MSE = mean_squared_error(np.log(Gold_Factor_data['GoldBar_Sale']),np.log(Prediction_df['Predict GoldBar_Sale price (Forward looking)']))
# Root Mean Squared Error (RMSE)
RMSE = np.sqrt(MSE)
print('MAE : ',MAE) # ค่ายิ่งต้ำยิ่งดี
print('MSE : ',MSE) # ค่ายิ่งต้ำยิ่งดี
print('RMSE : ',RMSE) # ค่ายิ่งต้ำยิ้งดี

Version 2

In [ ]:
# Gold_Factor_data[['LOGIT']+Factor_interested].corr()
Gold_Factor_data[['GoldBar_Sale']+Factor_interested].corr()

In [ ]:
Gold_Factor_data.plot(x='(Inflation)',y='GoldBar_Sale',style='o')
plt.ylabel('Gold Price')

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
X = Gold_Factor_data[['(Inflation)']]
y = np.log(Gold_Factor_data[['GoldBar_Sale']])

In [ ]:
# y = ax + b
lrm = LinearRegression()
lrm.fit(X,y)

In [ ]:
lrm.intercept_ # b

In [ ]:
lrm.coef_ # a

In [ ]:
predictions = lrm.predict(X)
predictions

In [ ]:
plt.scatter(X,y,color='black')
plt.plot(X,predictions)

In [ ]:
from sklearn import metrics
metrics.mean_absolute_error(y,predictions)

In [ ]:
Gold_Factor_data[['GoldBar_Sale']+Factor_interested].corr()

In [ ]:
from sklearn.linear_model import LinearRegression
X = Gold_Factor_data[Factor_interested]
y = np.log(Gold_Factor_data[['GoldBar_Sale']])

In [ ]:
lrm = LinearRegression()
lrm.fit(X,y)

In [ ]:
predictions = lrm.predict(X)
np.exp(predictions)

In [ ]:
from sklearn import metrics
metrics.mean_absolute_error(y,predictions) # MAE

In [ ]:
y.mean()

In [ ]:
metrics.mean_squared_error(y,predictions) # MSE

In [ ]:
np.sqrt(metrics.mean_squared_error(y,predictions)) #RMSE

In [ ]:
lrm.score(X,y) # R-Square

In [ ]:
# MEAN : 10.040206
# MAE : 0.09521426489602582 ค่าต่ำ แสดงถึง โมเดลคาดการณ์ได้ดี
# MSE : 0.014789767007600822 ค่าต่ำแสดงถึง โมเดลคาดการณ์ได้ดี
# RMSE : 0.12161318599395718 # ค่าต่ำแสดงถึง ค่าความคลาดเคลื่อนจากค่าจริง
# R-Squared : 0.532340099507458 # โมเดลสามารถอธิบายความแปรผันของตัวแปรตามได้ประมาณ 53.23% 
# ซึ่งแสดงให้เห็นว่ามีความสัมพันธ์ระหว่างตัวแปรต้นและตัวแปรตาม แต่ยังไม่สูงมากนัก อาจมีปัจจัยอื่นที่ยังไม่ได้ถูกพิจารณา.

OLS Regression Model

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
X = Gold_Factor_data[Factor_interested]
y = (Gold_Factor_data[['GoldBar_Sale']])

In [ ]:
model = sm.OLS(y,X)

In [ ]:
# model.params # ceof
result = model.fit()
result.params 

In [ ]:
result.summary()

Plot Graph

       ! Gold_data -> File


In [ ]:
np.log(Gold_data['GoldBar_Sale']).plot(figsize = (20,5) , legend = True)

In [ ]:
#Plot Two Side View
fig , ax = plt.subplots(1,1,figsize=(16,5) , dpi=120)
x = Gold_data['Report Date'].values

y1 = np.log(Gold_data['GoldBar_Sale'].values)

plt.fill_between( x , y1 = y1 , y2 = -y1 , alpha=0.9 , linewidth=2, color='skyblue')

plt.ylim(-700 , 700 )
plt.title('GoldBar_Sale (Two Side View) ' , fontsize = 20)

plt.hlines( y=0 , xmin = np.min(Gold_data['Report Date']) , xmax = np.max(Gold_data['Report Date']) , linewidth=0.5)

plt.show()

In [ ]:
#ทดสอบ Stationary
rolling_mean = Gold_data['GoldBar_Sale'].rolling(window = 12).mean()
rolling_std = Gold_data['GoldBar_Sale'].rolling(window = 12).std()
fig , ax = plt.subplots(1,1,figsize=(16,5),dpi = 120)
plt.plot(Gold_data['GoldBar_Sale'],color = 'blue',label = 'Original')
plt.plot(rolling_mean , color = 'red' , label = 'Rolling Mean')
plt.plot(rolling_std , color = 'black' , label = 'Rolling Std')
plt.legend(loc = 'best')
plt.title('Rolling Mean & Rolling Standard Deviation')
plt.show()

In [ ]:
result = adfuller(Gold_data['GoldBar_Sale'])
print('ADF Statistic : {}'.format(result[0]))
print('p-value : {}'.format(result[1]))
print('Critical Values:')
for key , value in result[4].items():
    print('\t{}: {}'.format(key,value))

In [ ]:
# function
def get_stationarity(timeseries):
    rolling_mean = timeseries.rolling(window = 12).mean()
    rolling_std = timeseries.rolling(window = 12).std()
    original = plt.plot(timeseries, color = 'blue' , label ='Original')
    mean = plt.plot(rolling_mean , color = 'red' , label = 'Rolling Mean')
    std = plt.plot(rolling_std , color = 'black' , label = 'Rolling Std')
    plt.legend(loc = 'best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block =False)
    # Dickey-Fuller Test
    result = adfuller(timeseries)
    print('ADF Statiestic: {} '.format(result[0]))
    print('p-value: {}'.format(result[1]))
    print('Critical Values:')
    for key , value in result[4].items():
        print('\t{} : {}'.format(key,value))

In [ ]:
# วิธีแรกในการปรับข้อมูลให้เรียบคือ การลบด้วยค่าเฉลี่ยเคลื่อนที่ 12 เดือนย้อนหลังออกไป เพื่อกำจัด ฤดูกาลกับแนวโน้ม
rolling_mean = Gold_data['GoldBar_Sale'].rolling(window=12).mean()
df_log_minus_mean = Gold_data['GoldBar_Sale'] - rolling_mean
df_log_minus_mean.dropna(inplace=True)
get_stationarity(df_log_minus_mean)

In [ ]:
# p value ต่ำแล้ว ลองวิธี take log
rolling_mean_exp_decay = Gold_data['GoldBar_Sale'].ewm(halflife=12,min_periods=0,adjust=True).mean()

df_log_exp_decay = Gold_data['GoldBar_Sale'] - rolling_mean_exp_decay
df_log_exp_decay.dropna(inplace = True)
get_stationarity(df_log_exp_decay)

In [ ]:
# วิธีลบด้วยค่าก่อนหน้า
df_log_shift = Gold_data['GoldBar_Sale'] - Gold_data['GoldBar_Sale'].shift()
df_log_shift.dropna(inplace = True)
get_stationarity(df_log_shift)

สร้าง ARIMA Model

In [ ]:
# import libaries
from dateutil.parser import parse
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pmdarima as pm
from pmdarima import model_selection
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
import joblib
import os

In [ ]:
# 1,1,2 ARIMA Model
model = pm.ARIMA(order=(1,1,2),seasonal=False)
model_fit = model.fit(Gold_data['GoldBar_Sale'],disp=0)
print(model_fit.summary())

In [ ]:
# 1,1,1 ARIMA Model
model = pm.ARIMA(order=(1,1,1),seasonal=False)
model_fit = model.fit(Gold_data['GoldBar_Sale'],disp=0)
print(model_fit.summary())

In [ ]:
# เลือกอันที่ P > |Z| ดีๆ คือค่าน้อยๆ
preds, conf_int = model_fit.predict(n_periods=12, return_conf_int=True)
residuals = pd.DataFrame(preds)
fig,ax = plt.subplots(1,2)
residuals.plot(title="Residuals",ax=ax[0])
residuals.plot(kind='kde',title='Density',ax=ax[1])
plt.show()

ARIMA MODEL pmdarima 

In [ ]:
arima = pm.auto_arima(Gold_data['GoldBar_Sale'], start_p=1, start_q=1, max_p=12, max_q=5, max_order = 12,
                     trace=False, suppress_warnings=True, stepwise = True,
                     error_action='ignore', m=12)
period_predict = 10
preds, conf_int = arima.predict(n_periods=12*period_predict, return_conf_int=True)
arima.summary()

In [ ]:
preds

In [ ]:
def eomonth(d, months=0):
    months = int(months)
    if isinstance(d, list):
        return [eomonth(d0, months) for d0 in d]
    elif isinstance(d, pd.Series):
        return d.map(lambda d0: eomonth(d0, months))
    else:
        y, m = divmod(d.month + months + 1, 12)
        # y, m = int(y), int(m)
        if m==0:
            y -= 1
            m = 12
        return pytz.UTC.localize(datetime.datetime(d.year + y, m, 1) - datetime.timedelta(days=1))

In [ ]:
Date = Gold_data['Report Date']
last_date = Date.iloc[-1]
arr = []
period = period_predict*12
for i in range(period):
    next_month = eomonth(last_date,i+1)
    arr = np.append(arr,next_month)

Date_forecast_period = pd.DataFrame(arr)
Date_forecast_period.columns = ['Report Date']
Date_forecast_period['Report Date'] = Date_forecast_period['Report Date'].dt.tz_localize(None)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(Gold_data['Report Date'] , Gold_data['GoldBar_Sale'] , label = 'Actual Data')
plt.plot(Date_forecast_period,preds , label='Forecast' , color = 'red')
# plt.fill_between(Date_forecast_period , conf_int[:,0],conf_int[:,1] , color='pink' , alpha=0.3)
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Actual Data vs Forecast')
plt.legend()
plt.grid(True)
plt.show()

Forecast GoldBar_Sale price in Thailand with Feature_Seletion by LinearRegression and ARIMA model